In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor
from qwen_vl_utils import process_vision_info


D:\anaconder\envs\dots_ocr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 本地模型路径（使用绝对路径更稳妥，示例路径需替换为你的实际路径）
model_path = "../weights/DotsOCR"  # Windows示例
# model_path = "/home/你的用户名/dots.ocr/weights/DotsOCR"  # Linux/Mac示例

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,  # 本地路径直接传入
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,  # 必须开启
    load_in_8bit=True
)

# 加载处理器
processor = AutoProcessor.from_pretrained(
    model_path,
    trust_remote_code=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not avail

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.22s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [16]:
# 使用项目自带的测试图片
image_path = "C:/Users/fsq/Desktop/1.png"

# 模型指令（固定格式）
prompt = """Please output the layout information from the PDF image, including each layout element's bbox, its category, and the corresponding text content within the bbox.

1. Bbox format: [x1, y1, x2, y2]
2. Layout Categories: ['Caption', 'Footnote', 'Formula', 'List-item', 'Page-footer', 'Page-header', 'Picture', 'Section-header', 'Table', 'Text', 'Title']
3. Text Extraction & Formatting Rules:
    - Picture: For the 'Picture' category, the text field should be omitted.
    - Formula: Format its text as LaTeX.
    - Table: Format its text as HTML.
    - All Others (Text, Title, etc.): Format their text as Markdown.
4. Constraints: Original text, sorted by reading order
5. Output: Single JSON object.
"""

In [17]:
# 构造输入
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": prompt}
        ]
    }
]

In [18]:
# 处理输入
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt"
).to(model.device)

In [19]:
# 生成结果
generated_ids = model.generate(** inputs, max_new_tokens=24000)
generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [20]:

# 打印结果
print(output_text)

[{"bbox": [26, 10, 1044, 42], "category": "Text", "text": "执行推理代码后, `print(output_text)`会在 Notebook 的“输出区”直接显示 JSON 格式的结果。"}, {"bbox": [31, 105, 1030, 135], "category": "List-item", "text": "* 找到所有\"category\"不是\"Picture\"的元素（图片无文字），直接读取对应的\"text\"字段即可。"}, {"bbox": [31, 163, 1200, 239], "category": "List-item", "text": "* 示例中：\"Title\"的文字是# Introduction to Machine Learning, \"Text\"的文字是正文内容，直接复制粘贴就能用。"}]
